In [2]:
import pandas as pd
from datetime import datetime
import altair as alt

from statistics import *

file = pd.read_csv("merged_stop_data.csv", dtype = str)
file = file[file["Bus"] != "0"]
file = file[file["Time"].notna()]
file = file[file["Date"].notna()]
file = file[file["Route"].isin(["Gold", "Silver", "Green"])]
# file = file[file["Route"].isin(["Silver", "Gold", "Green"])]

In [3]:
def calculate_distance(start_stop, destination_stop, hour, day):
    stop_filter = file[file["Stop"].isin([start_stop, destination_stop])]
    time_column = [datetime.strptime(f"{date}_{time}", "%m/%d/%Y_%H:%M:%S") for time, date in zip(stop_filter["Time"], stop_filter["Date"])]
    stop_filter = stop_filter.assign(Converted_Time=time_column)
    times = []
    for index, data in stop_filter.groupby("Bus"):
        previous_stop = ""
        previous_time = 0
        previous_route = ""
        # TODO why are there such big outliers /shrug. Just tried grouping by bus
        for current_stop, current_time, route in zip(data["Stop"], data["Converted_Time"], data["Route"]):

            correct_stop_order = current_stop == destination_stop and previous_stop == start_stop
            consistant_route = previous_route == route
            correct_hour = current_time.hour == hour
            correct_day = current_time.weekday() == day
            correct_month = 3

            if(correct_stop_order and consistant_route and correct_hour and correct_day and correct_month):
                time_difference = current_time - previous_time
                times.append(round(time_difference.seconds / 60, 2))
            previous_stop = current_stop
            previous_time = current_time
            previous_route = route
    
    filtered_time = [t for t in times if t < 60]

    if(len(filtered_time) == 0):
        return -1
    # print(f"Median at hour {hour} on day {day}:", median(filtered_time))
    return median(filtered_time)

In [4]:
from tqdm import tqdm
first_location = "Athletics Complex W"
second_location = "CRI Deck"
print(calculate_distance(first_location, second_location, 15, 6) / 2)
exit()
hour_time = []
hour = []
day = []
for i in range(7):
    for j in range(24):
        if j < 7:
            continue

        med = calculate_distance(first_location, second_location, j, i) / 2
        hour_time.append(med)
        hour.append(i)
        day.append(j)
        print(i, j, med)

2.2
0 7 2.035
0 8 2.05
0 9 2.14
0 10 2.125
0 11 2.165
0 12 2.285
0 13 2.365
0 14 2.465
0 15 2.4
0 16 2.295
0 17 2.465
0 18 2.385
0 19 2.4
0 20 2.365
0 21 2.385
0 22 2.8
0 23 1.975
1 7 2.015
1 8 2.115
1 9 2.3
1 10 2.165
1 11 2.3775000000000004
1 12 2.35
1 13 2.365
1 14 2.6
1 15 2.3075
1 16 2.355
1 17 2.525
1 18 2.415
1 19 2.44
1 20 2.615
1 21 2.8575
1 22 2.6550000000000002
1 23 2.225
2 7 2.085
2 8 2.05
2 9 2.215
2 10 2.15
2 11 2.285
2 12 2.385
2 13 2.485
2 14 2.5300000000000002
2 15 2.335
2 16 2.315
2 17 2.5324999999999998
2 18 2.46
2 19 2.3499999999999996
2 20 2.465
2 21 2.335
2 22 2.7225
2 23 2.96
3 7 2.035
3 8 2.1
3 9 2.295
3 10 2.235
3 11 2.335
3 12 2.335
3 13 2.3525
3 14 2.485
3 15 2.3075
3 16 2.325
3 17 2.66
3 18 2.5
3 19 2.5
3 20 2.46
3 21 2.59
3 22 2.415
3 23 2.26
4 7 2.035
4 8 2.105
4 9 2.2
4 10 2.2325
4 11 2.35
4 12 2.3925
4 13 2.465
4 14 2.5625
4 15 2.335
4 16 2.285
4 17 2.3875
4 18 2.44
4 19 2.36
4 20 2.525
4 21 2.44
4 22 2.7825
4 23 11.94
5 7 -0.5
5 8 -0.5
5 9 1.985
5 10 2.

In [ ]:
plot_frame = pd.DataFrame()
plot_frame["Hour"] = day
plot_frame["Minutes"] = hour_time
plot_frame["Day"] = hour
print(plot_frame)

alt.Chart(plot_frame).mark_bar().encode(
    x="Hour:N",
    y="Minutes:Q",
    # color=alt.Color("Day:N", scale=alt.Scale(scheme='yellowgreenblue')),
    column="Day"
).properties(
width=150
).interactive()